## Attributi sull'intero testo

- Dimensione del vocabolario (V)
- Lunghezza del testo in numero di parole (T)
- Rapporto V/T
- Entropia (H)

In [48]:
import math

def word_counter(RDD):
    '''
    Data una RDD, conta quante volte compare ogni parola ritornando anche la dimensione del vocabolario.
    
    Parameters
    ----------
    RDD : RDD
        RDD del file in input

    Returns
    -------
    (RDD, int)
        RDD dell'output del word count e dimensione del vocabolario
    '''
    
    word_counter = (RDD.flatMap(lambda x: x)
                    .map(lambda x: (x,1))
                    .reduceByKey(lambda a,b: a+b)
                    .sortBy(lambda x: -x[1])
                   )
    
    return word_counter, word_counter.count()

def text_length_in_words(RDD_word_counter):
    '''
    Calcola la lunghezza del testo in termini di numero di parole.
    
    Parameters
    ----------
    RDD_word_counter : RDD
        RDD dell'output del word count

    Returns
    -------
    int
        numero di parole totali presenti nel testo
    '''

    return (RDD_word_counter.map(lambda x: x[1])
                            .reduce(lambda a,b: a+b)
           )

def entropy(RDD_word_counter, text_len):
    '''
    Calcola l'entropia (numero medio di bit richiesti per rappresentare tutte le parole del testo).
    
    Parameters
    ----------
    RDD_word_counter : RDD
        RDD dell'output del word count
    text_len : int
        numero di parole totali presenti nel testo

    Returns
    -------
    float
        valore dell'entropia
    '''
    
    return -(RDD_word_counter.map(lambda x: (x[1]/text_len) * math.log2(x[1]/text_len))
            .reduce(lambda a,b: a+b)
            )    # l'entropia ha segno negativo

## Attributi sulle frasi

- Lunghezza massima di una frase (MSL)
- Lunghezza media di una frase (ASL)
- Lunghezza minima di una frase (mSL)

In [49]:
def sentence_lengths(RDD):
    '''
    Ritorna una lista che contiene le lunghezze (in termini di numero di parole) di tutte le frasi di un testo.
    
    Parameters
    ----------
    RDD : RDD
        RDD del file in input

    Returns
    -------
    list
        lunghezze delle frasi
    '''
    
    # OPERAZIONI PRELIMINARI SUL TESTO
    text = RDD.flatMap(lambda x: x).reduce(lambda a,b: a + ' ' + b) # METTO TUTTO IL TESTO IN UNA STRINGA UNICA
    text = text.replace("?", ".") # ? TERMINA UNA FRASE
    text = text.replace("!", ".") # ! TERMINA UNA FRASE
    text = text.split('. ') # SPLITTO QUANDO TROVO UN CARATTERE CHE TERMINA UNA FRASE (. SEGUITO DA UNO SPAZIO)
    
    return (sc.parallelize(text)
            .map(lambda x: len(x.split(' ')))
            .collect()
           ) # PER OGNI FRASE TROVATA CONTO LE SUE PAROLE

# MAIN

In [50]:
def remove_number_some_punctuation_marks(row):

    lowercase = row.lower()
    lowercase = lowercase.replace("--", " ")
    
    res = ""
    
    for char in lowercase:
        if not ('0' <= char <= '9' or char == '"'):
            res += char

    return res

def remove_number_punctuation_marks(row):
    
    lowercase = row.lower()
    lowercase = lowercase.replace("--", " ")
    
    res = ""
    
    for char in lowercase:
        if 'a' <= char <= 'z' or char == ' ' or char == '-' or char == "'":
            res += char

    return res

def load_file_without_punctuations_marks(filepath):
    # caricamento del dataset
    raw_text = sc.textFile(filepath)

    # rimuoviamo i numeri e i segni di punteggiatura
    
    return (raw_text.filter(bool)                    # rimuoviamo le stringhe vuote
        .map(remove_number_punctuation_marks)
        .map(lambda x : ' '.join(x.split()))        # rimuoviamo diversi spazi bianchi con uno
        .map(lambda row : row.split(" "))
       )

def load_file_without_number(filepath):
    # caricamento del dataset
    raw_text = sc.textFile(filepath)

    # rimuoviamo i numeri e i segni di punteggiatura
    
    return (raw_text.filter(bool)                    # rimuoviamo le stringhe vuote
        .map(remove_number_some_punctuation_marks)
        .map(lambda x : ' '.join(x.split()))        # rimuoviamo diversi spazi bianchi con uno
        .map(lambda row : row.split(" "))
       )


data = load_file_without_punctuations_marks("datasets/Anthony Trollope___The O'Conors of Castle Conor from Tales from all Countries.txt")
data.persist()

PythonRDD[320] at RDD at PythonRDD.scala:53

In [51]:
RDD_word_counter, vocabulary_size = word_counter(data)
print("V:\t", vocabulary_size)

text_length = text_length_in_words(RDD_word_counter)
print("T:\t", text_length)

print("V/T:\t", vocabulary_size/text_length)

entropy_value = entropy(RDD_word_counter, text_length)
print("H:\t", entropy_value)

V:	 1558
T:	 7653
V/T:	 0.20358029530902913
H:	 8.605861060321123


In [52]:
sentences_data = load_file_without_number("datasets/Anthony Trollope___The O'Conors of Castle Conor from Tales from all Countries.txt")
sentences_data.persist()

PythonRDD[336] at RDD at PythonRDD.scala:53

In [54]:
lengths = sentence_lengths(sentences_data)
print("MSL:\t", max(lengths))
print("ASL:\t", sum(lengths)/len(lengths))
print("mSL:\t", min(lengths))

MSL:	 84
ASL:	 14.862135922330097
mSL:	 1
